In [ ]:
!sudo apt install tesseract-ocr
!pip install pytesseract

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 49 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 4,816 kB in 0s (11.5 MB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debco

In [ ]:
import os
import pytesseract
import random
try:
    from PIL import Image
except ImportError:
    import Image

In [ ]:
from google.colab import files

uploaded = files.upload()

Saving 1727056685.jpg to 1727056685.jpg
Saving Screenshot-2024-10-01-at-1.49.21 PM.jpg to Screenshot-2024-10-01-at-1.49.21 PM.jpg
Saving Telecom.jpg to Telecom.jpg


**DocQA using Pretrained Donut Model**

In [ ]:
import re

from transformers import DonutProcessor, VisionEncoderDecoderModel
import torch

processor = DonutProcessor.from_pretrained("naver-clova-ix/donut-base-finetuned-docvqa")
model = VisionEncoderDecoderModel.from_pretrained("naver-clova-ix/donut-base-finetuned-docvqa")

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/359 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/535 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/1.30M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.01M [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/478 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.74k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/803M [00:00<?, ?B/s]

Config of the encoder: <class 'transformers.models.donut.modeling_donut_swin.DonutSwinModel'> is overwritten by shared encoder config: DonutSwinConfig {
  "attention_probs_dropout_prob": 0.0,
  "depths": [
    2,
    2,
    14,
    2
  ],
  "drop_path_rate": 0.1,
  "embed_dim": 128,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 1024,
  "image_size": [
    2560,
    1920
  ],
  "initializer_range": 0.02,
  "layer_norm_eps": 1e-05,
  "mlp_ratio": 4.0,
  "model_type": "donut-swin",
  "num_channels": 3,
  "num_heads": [
    4,
    8,
    16,
    32
  ],
  "num_layers": 4,
  "patch_size": 4,
  "path_norm": true,
  "qkv_bias": true,
  "transformers_version": "4.46.3",
  "use_absolute_embeddings": false,
  "window_size": 10
}

Config of the decoder: <class 'transformers.models.mbart.modeling_mbart.MBartForCausalLM'> is overwritten by shared decoder config: MBartConfig {
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_cross_attention": true,
  "add

VisionEncoderDecoderModel(
  (encoder): DonutSwinModel(
    (embeddings): DonutSwinEmbeddings(
      (patch_embeddings): DonutSwinPatchEmbeddings(
        (projection): Conv2d(3, 128, kernel_size=(4, 4), stride=(4, 4))
      )
      (norm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): DonutSwinEncoder(
      (layers): ModuleList(
        (0): DonutSwinStage(
          (blocks): ModuleList(
            (0-1): 2 x DonutSwinLayer(
              (layernorm_before): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
              (attention): DonutSwinAttention(
                (self): DonutSwinSelfAttention(
                  (query): Linear(in_features=128, out_features=128, bias=True)
                  (key): Linear(in_features=128, out_features=128, bias=True)
                  (value): Linear(in_features=128, out_features=128, bias=True)
                  (dropout): Dropout(p=0.0, inplace=False)
           

In [ ]:
image = [Image.open('/content/1727056685.jpg'), Image.open('/content/Screenshot-2024-10-01-at-1.49.21 PM.jpg'), Image.open('/content/Telecom.jpg')]

In [ ]:
for i,img in enumerate(image,start=1):

  print(f'\n\nExtracted Info for Image_{i} \n')
  # List of questions
  questions = [
      "What is the Invoice Number or Invoice No.?",
      "What is the Invoice Date?",
      "What is the Customer Name or Bill To?",
      "What is the Total Amount or Invoice Due Amount?"
  ]

  # Prepare decoder inputs for multiple questions
  task_prompt = "<s_docvqa><s_question>{user_input}</s_question><s_answer>"
  pixel_values = processor(img, return_tensors="pt").pixel_values
  results = []

  for question in questions:
      prompt = task_prompt.replace("{user_input}", question)
      decoder_input_ids = processor.tokenizer(prompt, add_special_tokens=False, return_tensors="pt").input_ids

      # Generate outputs for each question
      outputs = model.generate(
          pixel_values.to(device),
          decoder_input_ids=decoder_input_ids.to(device),
          max_length=model.decoder.config.max_position_embeddings,
          pad_token_id=processor.tokenizer.pad_token_id,
          eos_token_id=processor.tokenizer.eos_token_id,
          use_cache=True,
          bad_words_ids=[[processor.tokenizer.unk_token_id]],
          return_dict_in_generate=True,
      )

      # Decode and process the result
      sequence = processor.batch_decode(outputs.sequences)[0]
      sequence = sequence.replace(processor.tokenizer.eos_token, "").replace(processor.tokenizer.pad_token, "")
      sequence = re.sub(r"<.*?>", "", sequence, count=1).strip()  # Remove first task start token

      # Convert to JSON (or any structured format) and store the result
      answer = processor.token2json(sequence)
      results.append(answer)

  # Print all results
  for result in results:
      print(result)




Extracted Info for Image_0 

{'question': 'What is the Invoice Number or Invoice No.?', 'answer': '22pa61y00005001'}
{'question': 'What is the Invoice Date?', 'answer': '09/09/2012'}
{'question': 'What is the Customer Name or Bill To?', 'answer': 'akanksha'}
{'question': 'What is the Total Amount or Invoice Due Amount?', 'answer': 'one hundred twenty four rupees and ninety six paisa only'}


Extracted Info for Image_1 

{'question': 'What is the Invoice Number or Invoice No.?', 'answer': '24wchgbb00002615'}
{'question': 'What is the Invoice Date?', 'answer': '30/06/2014'}
{'question': 'What is the Customer Name or Bill To?', 'answer': 'sarang s'}
{'question': 'What is the Total Amount or Invoice Due Amount?', 'answer': 'three hundred fifty eight rupees and twenty six paisa only'}


Extracted Info for Image_2 

{'question': 'What is the Invoice Number or Invoice No.?', 'answer': 'gst-6290'}
{'question': 'What is the Invoice Date?', 'answer': '26-01-2020'}
{'question': 'What is the Cus

**DocQA using Pretrained LayoutLM Model**

In [ ]:
from transformers import pipeline

nlp = pipeline(
    "document-question-answering",
    model="impira/layoutlm-document-qa",
)


for i,img in enumerate(image,start=1):

  print(f'\n\n Extracted Info for Image_{i}\n')

  Invoice_Number = nlp(
      img,
      "What is the Invoice Number or Invoice No.?"
  )

  Invoice_Date = nlp(
      img,
      "What is the Invoice Date?"
  )

  Customer_Name = nlp(
      img,
      "What is the Customer Name or Bill To?"
  )

  Total_Amount = nlp(
      img,
      "What is the Total Amount/Value in Invoice ?"
  )
  print(f'Invoice Number: {Invoice_Number[0]["answer"]}')
  print(f'Invoice Date: {Invoice_Date[0]["answer"]}')
  print(f'Customer Name: {Customer_Name[0]["answer"]}')
  print(f'Total Amount: {Total_Amount[0]["answer"]}')




 Extracted Info for Image_1

Invoice Number: 22PA6LLJ00005001
Invoice Date: 09/09/2022
Customer Name: Akanksha
Total Amount: 124.96


 Extracted Info for Image_2

Invoice Number: 24WCHGBB00002615
Invoice Date: 30/06/2024
Customer Name: Sarang S
Total Amount: 341.2


 Extracted Info for Image_3

Invoice Number: :GST-6290
Invoice Date: 26-01-2020
Customer Name: IDFC FIRST BANK LTD (FINANCE)
Total Amount: 69,990.00
